<a href="https://colab.research.google.com/github/herjh0405/DACON_Meal/blob/master/%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9_%EC%8B%9D%EC%88%98_%EC%9D%B8%EC%9B%90_%EC%98%88%EC%B8%A1_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DACON LH_구내식당 식수 인원 예측
* 가설 검증 및 시각화

## 환경 설정

### Model Install

In [ ]:
!pip install pycaret
!pip install konlpy

### Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import Library

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import datetime
import itertools
np.random.seed(0)

from pycaret.regression import *
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss

from tqdm.notebook import tqdm
from konlpy.tag import Kkma

import torch
import os, re

In [ ]:
# 한글 폰트 사용
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os

def change_matplotlib_font(font_download_url):
    FONT_PATH = 'MY_FONT'
    
    font_download_cmd = f"wget {font_download_url} -O {FONT_PATH}.zip"
    unzip_cmd = f"unzip -o {FONT_PATH}.zip -d {FONT_PATH}"
    os.system(font_download_cmd)
    os.system(unzip_cmd)
    
    font_files = fm.findSystemFonts(fontpaths=FONT_PATH)
    for font_file in font_files:
        fm.fontManager.addfont(font_file)

    font_name = fm.FontProperties(fname=font_files[0]).get_name()
    matplotlib.rc('font', family=font_name)
    print("font family: ", plt.rcParams['font.family'])

font_download_url = "https://fonts.google.com/download?family=Noto%20Sans%20KR"
change_matplotlib_font(font_download_url)

## Data Preprocessing

### Load Data

In [ ]:
path = '/content/drive/MyDrive/DACON/Dacon_Industry_Meal/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')

train.columns = ['일자', '요일', '정원','휴가자', '출장자', '야근자',\
                 '재택근무자', '조식', '중식', '석식', '중식계', '석식계']
test.columns = ['일자', '요일', '정원','휴가자', '출장자', '야근자',\
                 '재택근무자', '조식', '중식', '석식']

In [ ]:
train[['재택근무자', '중식계', '석식계']] = train[['재택근무자', '중식계', '석식계']].astype('int')
test['재택근무자'] = test['재택근무자'].astype('int')

def df_date(df) :
    df['일자'] = pd.to_datetime(df['일자'])
    df['년'] = df['일자'].dt.year
    df['월'] = df['일자'].dt.month
    df['일'] = df['일자'].dt.day
    df['월일'] = df['일자'].apply(lambda x : str(x)[5:10])

    return df

train = df_date(train)
test = df_date(test)

## 가설 수립 및 검정

### 가설 1. 재택근무는 코로나 때문에 생겼을 것이다.
* 이후에, 코로나 확진자 수와 연동할 것이지만, 현재로써도 가설이 맞음을 확인할 수 있다.
![image](https://user-images.githubusercontent.com/54921730/120912709-9330a980-c6cc-11eb-97e5-52ae39115551.png)
* 이제 여러가지 의문점이 추가로 등장
    * 재택 근무자 수가 증가함에 따라 식수 인원도 증가하는가?
        * 의외의 결과 발생 -> 코로나 환자 증가로 인해, 외식이 어려워지면서 식수 인원이 증가한 것으로 추측
            * 그렇다면 감시 단계 데이터도 연동해야하는가?  
![image](https://user-images.githubusercontent.com/54921730/120911123-8e192d80-c6bf-11eb-9146-1a52356cf9d1.png)
![image](https://user-images.githubusercontent.com/54921730/120911048-f87d9e00-c6be-11eb-9f3c-85d39d82a74d.png)
    * 그렇다면 코로나 전후로 데이터를 구분해야하는 것은 아닌가?

### 결론
* 위 여러 의문점들이 남아있지만, 아직 명쾌한 해답이 나오지 않았음.
* 컴퓨터가 학습하기 너무 어려운 데이터라 판단, 데이터에서 제외

## 국내 일일 코로나 확진자 추이

In [ ]:
parse

In [ ]:
# 코로나 발생현황 데이터 불러와서 비교할 예정
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.parse import unquote, quote_plus, urlencode

# 1. URL 파라미터 분리하기.
# Service URL
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
servicekey = '9qhUwKCc9ASN8d7C8HnGWLyQOr1P4rkXKxsYwShxM6PLhVSnNwqlkZQsY%2Fk%2Bx4Rixmyznbm65TySlJtMbVi3KA%3D%3D'
decodekey = unquote(servicekey)


startCreateDt = 20190101
endCreateDt = 20220101

query_params = '?' + urlencode({
    quote_plus('serviceKey'): decodekey,
    quote_plus('startCreateDt'): startCreateDt,
    quote_plus('endCreateDt'): endCreateDt
})

URL = url + query_params

request = Request(URL)
page = urlopen(request).read()
soup = BeautifulSoup(page, 'html.parser')
parse = soup.find_all(['decidecnt', 'statedt', 'examcnt'])

co_df = []
for dt, cnt, care in zip(parse[2::3], parse[0::3], parse[1::3]):
    co_df.append([dt.text, int(cnt.text), int(care.text)])

co_df = pd.DataFrame(co_df, columns=['일자', '누적확진자', '일일검사자'])
co_df = co_df.sort_values(by='일자').reset_index(drop=True)
co_df['누적확진자'][4] = 15
co_df['일일확진자'] = co_df['누적확진자'].diff()
co_df = co_df.loc[1:].reset_index(drop=True)  # 맨 앞의 데이터는 지우기 (일별 확진자가 계산되지 않았기 때문에)
co_df['일자'] = pd.to_datetime(co_df['일자'])
co_df['누적검사자'] = list(itertools.accumulate(co_df['일일검사자']))

### 진주시 일일 코로나 확진자 데이터 크롤링

In [ ]:
import ssl
context = ssl._create_unverified_context()
weekday_list = ['(월)', '(화)', '(수)', '(목)', '(금)', '(토)', '(일)']
jinju_co = [[], []]

for i in tqdm(range(2, 6)):
    request= Request('https://www.jinju.go.kr/05190/05641.web?&cpage='+str(i))
    page = urlopen(request, context=context).read()
    soup = BeautifulSoup(page, 'html.parser')
    for i in soup.select('span.t1') : 
        if ('진주' in i.text) and ('경남' in i.text) :
            jinju_co[0].append(i.text)
        if  i.text[-3:] in weekday_list : 
            jinju_co[1].append(i.text)

In [ ]:
jinju_df = pd.DataFrame([jinju_co[0], jinju_co[1][1:]]).T
jinju_df.columns = ['번호', '일자']
jinju_df['요일'] = jinju_df['일자'].apply(lambda x : x[-3:].replace('(','').replace(')',''))
jinju_df['일자'] = jinju_df['일자'].apply(lambda x: x[:-4].replace(' ','').replace('.', '-'))
jinju_df['일자'] = jinju_df.apply(lambda x : '2020-'+x['일자'] if x.name > 759\
                                else '2021-'+x['일자'], axis=1)
jinju_df['일자'] = pd.to_datetime(jinju_df['일자'])
# 토요일에 확진자가 발생했을 경우 금요일로, 일요일에 확진자가 발생했을 경우 월요일로 변경
jinju_df['일자'] = jinju_df.apply(lambda x : x['일자'] - datetime.timedelta(1) if x['요일']=='토'
                            else (x['일자'] + datetime.timedelta(1) if x['요일']=='일'
                            else x['일자']), axis=1)
jinju_df = jinju_df[jinju_df['번호']!='진주기타1(경남기타1)']
jinju_df['번호'] = jinju_df['번호'].apply(lambda x : x.split('(')[0].replace('진주', '')).astype('int')
jinju_df = jinju_df.groupby(['일자'])['번호'].max().reset_index()
jinju_df.columns = ['일자', '진주_누적확진자']
jinju_df.sort_values('일자', inplace=True)
jinju_df['진주_일일확진자'] = jinju_df['진주_누적확진자'].diff()
jinju_df['진주_일일확진자'][0] = 2
kojin_df = pd.merge(co_df, jinju_df, on='일자', how='left').iloc[:434]
kojin_df['진주_일일확진자'] = kojin_df['진주_일일확진자'].fillna(0)
kojin_df['진주_누적확진자'] = list(itertools.accumulate(kojin_df['진주_일일확진자']))

### 경남 코로나 일일 확진자

In [ ]:
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
servicekey = '9qhUwKCc9ASN8d7C8HnGWLyQOr1P4rkXKxsYwShxM6PLhVSnNwqlkZQsY%2Fk%2Bx4Rixmyznbm65TySlJtMbVi3KA%3D%3D'
decodekey = unquote(servicekey)


startCreateDt = 20200201
endCreateDt = 20210409

query_params = '?' + urlencode({
    quote_plus('serviceKey'): decodekey,
    quote_plus('startCreateDt'): startCreateDt,
    quote_plus('endCreateDt'): endCreateDt
})

URL = url + query_params

request = Request(URL)
page = urlopen(request).read()
soup = BeautifulSoup(page, 'html.parser')
import time
st_time = time.time()
sido_df = []
check_point = 0
sido_corona = soup.find_all(['gubun','stdday', 'defcnt','localocccnt'])
for i in range(len(sido_corona)) : 
    if '경남' in sido_corona[i].text : 
        sido_df.append([sido_corona[i-2].text, sido_corona[i-1].text])
    
    if i%1000 == 0 : 
        ed_time = time.time() - st_time
        print(f'\r남은 데이터 : {i}/{len(sido_corona)}    남은 시간 : {ed_time*(len(sido_corona)-i)}', end='')

In [ ]:
kn_df = pd.DataFrame(sido_df, columns=['일자', '경남_누적확진자'])
kn_df['일자'] = kn_df['일자'].apply(lambda x: x[:-4].replace(' ','').replace('년', '-').replace('월', '-').replace('일', ''))
kn_df = kn_df.iloc[:-1]
kn_df['일자'] = pd.to_datetime(kn_df['일자'])

## 총 데이터 결합

In [ ]:
corona_df = pd.merge(kojin_df, kn_df, on='일자', how='left')
corona_df = corona_df.fillna(0)
corona_df['경남_누적확진자'][20:32] = [2, 4, 7, 17, 23, 28, 43, 48, 58, 59, 61, 62]
corona_df['경남_누적확진자'] = corona_df['경남_누적확진자'].astype('int')
corona_df['경남_일일확진자'] = corona_df['경남_누적확진자'].diff()
corona_df.dropna(inplace=True)
corona_df.iloc[:, 1:] = corona_df.iloc[:, 1:].astype(int)

In [ ]:
corona_df.to_csv('./국내_진주_경남-누적_일일확진자.csv', index=False)

## 원본데이터+코로나 데이터

In [ ]:
co_train = pd.merge(train, corona_df, on='일자', how='left')
co_train = co_train.fillna(0)

In [ ]:
co_graph = co_train.iloc[900:]

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(co_graph['일자'], co_graph['재택근무자'], label='재택근무자')
plt.plot(co_graph['일자'], co_graph['일일확진자'], label='일일확진자')
plt.plot(co_graph['일자'], co_graph['진주_일일확진자'], label='진주_일일확진자')
plt.plot(co_graph['일자'], np.log2(co_graph['누적확진자']), label='누적확진자')
plt.plot(co_graph['일자'], co_graph['진주_누적확진자'], label='진주_누적확진자')
plt.plot(co_graph['일자'], co_graph['경남_일일확진자'], label='경남_일일확진자')
plt.plot(co_graph['일자'], co_graph['경남_누적확진자'], label='경남_누적확진자')
plt.plot(co_graph['일자'], co_graph['일일검사자']/100, label='일일검사자')
plt.plot(co_graph['일자'], np.log2(co_graph['누적검사자']), label='누적검사자')
plt.xlabel('일자')
plt.ylabel('인원 수')
plt.legend('best', labels=['재택근무자', '일별확진자', '진주_일일확진자',\
                           '누적확진자', '진주_누적확진자', '경남_일일확진자',\
                           '경남_누적확진자', '일일검사자', '누적검사자'])
plt.title('재택근무/일별확진자 수 추이')
plt.show()

In [ ]:
co_train.corr()

In [ ]:
import seaborn as sns
plt.figure(figsize=(8, 6))
sns.heatmap(co_train.corr())
plt.show()

In [ ]:
train_18 = train.loc[train['년']==2018]
train_19 = train.loc[train['년']==2019]
train_20 = train.loc[train['년']==2020]

In [ ]:
plt.plot(train.groupby(['년'])['정원'].mean())
plt.title('년도 별 정원', fontsize=12)

plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(121)
bar_width = 0.3
index = train_19.groupby(['월'])['중식계'].sum().index
alpha = 0.5

plt.bar(index-bar_width, train_18.groupby(['월'])['중식계'].sum(),
        label='2018', width=0.3, alpha=alpha)
plt.bar(index, train_19.groupby(['월'])['중식계'].sum(), label='2019', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['중식계'].sum(), label='2020', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['재택근무자'].sum(), label='재택근무자',
        bottom=train_20.groupby(['월'])['중식계'].sum(), width=0.3, alpha=alpha)

plt.xticks(index, label=index)
plt.xlabel('월')
plt.ylabel('중식계 총합')
plt.legend('best', labels=['2018', '2019', '2020', '재택근무자'])
plt.title('코로나가 중식계에 끼친 영향', fontsize=12)

plt.subplot(122)
bar_width = 0.3
index = train_19.groupby(['월'])['석식계'].sum().index
alpha = 0.5

plt.bar(index-bar_width, train_18.groupby(['월'])['석식계'].sum(),
        label='2018', width=0.3, alpha=alpha)
plt.bar(index, train_19.groupby(['월'])['석식계'].sum(), label='2019', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['석식계'].sum(), label='2020', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['재택근무자'].sum(), label='재택근무자',
        bottom=train_20.groupby(['월'])['석식계'].sum(), width=0.3, alpha=alpha)

plt.xticks(index, label=index)
plt.xlabel('월')
plt.ylabel('석식계 총합')
plt.legend('best', labels=['2018', '2019', '2020', '재택근무자'])
plt.title('코로나가 석식계에 끼친 영향', fontsize=12)
plt.show()